<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M4_timeseries_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install darts

In [ ]:
# !pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys

import seaborn as sns
import altair as alt

In [ ]:
# before starting, we define some constants
figsize = (9, 6)
lowest_q, low_q, high_q, highest_q = 0.01, 0.1, 0.9, 0.99
label_q_outer = f"{int(lowest_q * 100)}-{int(highest_q * 100)}th percentiles"
label_q_inner = f"{int(low_q * 100)}-{int(high_q * 100)}th percentiles"

# Introduction to deep learning (transformer-based) Timeseries Forecast

* NeuralProphet (somewhat)
* **[N-BEATS (ElementAI)](https://arxiv.org/abs/1905.10437):** Essentially, N-BEATS is a pure deep learning architecture based on a deep stack of ensembled feed forward networks that are also stacked by interconnecting backcast and forecast links.
  * Easy to use: The model is simple to understand and has a modular structure (blocks and stacks). 
  * Multiple time-series: The model has the ability to generalize on many time-series.
* **[N-HITS (ElementAI)]():** XXXXXX
  * XXX
* **[DeepAR (Amazon)](https://arxiv.org/abs/1704.04110?context=stat.ML):** A novel time series model that combines both deep-learning and autoregressive characteristics. 
  * Multiple time series: DeepAR works really well with multiple time series: A global model is built by using multiple time series with slightly different distributions.
  * Rich set of inputs: Apart from historical data, DeepAR also allows the use of known future time sequences (a characteristic of auto-regressive models) and extra static attributes for series.
  * Automatic scaling: In DeepAR, there is no need to do that manually since the model under the hood scales the autoregressive input.
* **[Spacetimeformer:](https://arxiv.org/abs/2109.12218)** Considers both temporal and spatial relationships.
  * Interesting when dealing with geospatial data, but I have little experience there.
* **[Temporal Fusion Transformer](https://arxiv.org/abs/1912.09363):** Temporal Fusion Transformer (TFT) is a transformer-based time series forecasting model published by Google.
  * Multiple time series: Like the aforementioned models, TFT supports building a model on multiple, heterogeneous time series.
  * Rich number of features: TFT supports 3 types of features: i) time-dependent data with known inputs into the future ii) time-dependent data known only up to the present and iii) categorical/static variables, also known as time-invariant features. 
  * Interpretability: TFT gives much emphasis on interpretability. Specifically, by taking advantage of the Variable Selection component, the model can successfully measure the impact of each feature.
  * Prediction Intervals: Similar to DeepAR, TFT outputs a prediction interval along with the predicted values, by using quantile regression.

# Temporal Fusion Transformers

## Introduction

Temporal Fusion Transformer (TFT) is an **attention-based Deep Neural Network**, optimized for great performance and interpretability. 

**Advantages and novelties:**

* Rich features: 
  1. temporal data with known inputs into the future 
  2. temporal data known only up to the present and 
  3. exogenous categorical/static variables, also known as time-invariant features.
* Heterogeneous time series: Supports training on multiple time series, splits processing into 2 parts: local processing which focuses on the characteristics of specific events and global processing which captures the collective characteristics of all time series.
* Multi-horizon forecasting: Supports multi-step predictions. Apart from the actual prediction, TFT also outputs prediction intervals, by using the quantile loss function.
* Interpretability: At its core, TFT is a transformer-based architecture. By taking advantage of self-attention, this model presents a novel Muti Head attention mechanism which when analyzed, provides extra insight on feature importances. 

## Timesieries implementation (DARTS)

[Darts](https://unit8co.github.io/darts/) is a Python library for easy manipulation and forecasting of time series. It contains a variety of models, from classics such as ARIMA to deep neural networks. The models can all be used in the same way, using fit() and predict() functions, similar to scikit-learn. The library also makes it easy to backtest models, combine the predictions of several models, and take external data into account. 

Darts supports both univariate and multivariate time series and models. The ML-based models can be trained on potentially large datasets containing multiple time series, and some of the models offer a rich support for probabilistic forecasting.

While there is also a standalone version of TFT (eg. [standalone pytorch implementation](https://pypi.org/project/tft-torch/)), we will for this example use the Darts implementation, since it eases the integration of TFT in your traditional forecasting pipeline.

In [ ]:
from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.metrics import mape, rmse

from darts.utils.statistics import check_seasonality, plot_acf
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression

import warnings
warnings.filterwarnings("ignore")

Darts’ TFTModel incorporates the following main components from the original Temporal Fusion Transformer (TFT) architecture:

*gating mechanisms: skip over unused components of the model architecture
* variable selection networks: select relevant input variables at each time step.
* temporal processing of past and future input with LSTMs (long short-term memory)
* multi-head attention: captures long-term temporal dependencies
* prediction intervals: per default, produces quantile forecasts instead of deterministic values

### Training

TFTModel can be trained with past and future covariates. It is trained sequentially on fixed-size chunks consisting of an encoder and a decoder part:

* encoder: past input with input_chunk_length
  * past target: mandatory
  * past covariates: optional
* decoder: future known input with output_chunk_length
  * future covariates: mandatory (if none are available, consider TFTModel’s optional arguments add_encoders or add_relative_index from here)

In each iteration, the model produces a quantile prediction of shape (output_chunk_length, n_quantiles) on the decoder part.

### Forecast

Per default, TFTModel produces probabilistic quantile forecasts using QuantileRegression. This gives the range of likely target values at each prediction step. Most deep learning models in Darts’ - including TFTModel - support QuantileRegression and 16 other likelihoods to produce probabilistic forecasts by setting likelihood=MyLikelihood() at model creation.

### Toy example (Air Passangers)

Adopted from the [DARTS pakage tutorial](https://unit8co.github.io/darts/examples/13-TFT-examples.html)



This data set that is highly dependent on covariates. Knowing the month tells us a lot about the seasonal component, whereas the year determines the effect of the trend component.

Additionally, let’s convert the time index to integer values and use them as covariates as well.

All of the three covariates are known in the future, and can be used as future_covariates with the TFTModel.

In [ ]:
# Read data
from darts.datasets import AirPassengersDataset

series = AirPassengersDataset().load()

In [ ]:
series.head()

In [ ]:
# we convert monthly number of passengers to average daily number of passengers per month
series = series / TimeSeries.from_series(series.time_index.days_in_month)
series = series.astype(np.float32)

In [ ]:
# Create training and validation sets:
training_cutoff = pd.Timestamp("19571201")
train, val = series.split_after(training_cutoff)

In [ ]:
# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

In [ ]:
# create year, month and integer index covariate series
covariates = datetime_attribute_timeseries(series, attribute="year", one_hot=False)

In [ ]:
covariates = covariates.stack(datetime_attribute_timeseries(series, attribute="month", one_hot=False))

In [ ]:
covariates = covariates.stack(
    TimeSeries.from_times_and_values(
        times=series.time_index,
        values=np.arange(len(series)),
        columns=["linear_increase"],
    )
)

covariates = covariates.astype(np.float32)

In [ ]:
cov_train, cov_val = covariates.split_after(training_cutoff)

In [ ]:
# transform covariates (note: we fit the transformer on train split and can then transform the entire covariates series)
scaler_covs = Scaler()
scaler_covs.fit(cov_train)
covariates_transformed = scaler_covs.transform(covariates)

The TFTModel can only be used if some future input is given. Optional parameters add_encoders and add_relative_index can be useful, especially if we don’t have any future input available. They generate endoded temporal data is used as future covariates.

Since we already have future covariates defined in our example they are commented out.

In [ ]:
num_samples = 200
input_chunk_length = 24
forecast_horizon = 12

In [ ]:
my_model = TFTModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=forecast_horizon,
    hidden_size=64,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=16,
    n_epochs=300,
    add_relative_index=False,
    add_encoders=None,
    likelihood=QuantileRegression(
        # quantiles= [ 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
    ),  # QuantileRegression is set per default
    # loss_fn=MSELoss(),
    random_state=42,
)

In what follows, we can just provide the whole covariates series as future_covariates argument to the model; the model will slice these covariates and use only what it needs in order to train on forecasting the target train_transformed:

In [ ]:
my_model.fit(train_transformed, future_covariates=covariates_transformed, verbose=True)

We perform a one-shot prediction of 24 months using the “current” model - i.e., the model at the end of the training procedure:

In [ ]:
def eval_model(model, n, actual_series, val_series):
    pred_series = model.predict(n=n, num_samples=num_samples)

    # plot actual series
    plt.figure(figsize=figsize)
    actual_series[: pred_series.end_time()].plot(label="actual")

    # plot prediction with quantile ranges
    pred_series.plot(
        low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer
    )
    pred_series.plot(low_quantile=low_q, high_quantile=high_q, label=label_q_inner)

    plt.title("MAPE: {:.2f}%".format(mape(val_series, pred_series)))
    plt.legend()

In [ ]:
eval_model(my_model, 24, series_transformed, val_transformed)

Let’s backtest our TFTModel model, to see how it performs with a forecast horizon of 12 months over the last 3 years:

In [ ]:
backtest_series = my_model.historical_forecasts(
    series_transformed,
    future_covariates=covariates_transformed,
    start=train.end_time() + train.freq,
    num_samples=num_samples,
    forecast_horizon=forecast_horizon,
    stride=forecast_horizon,
    last_points_only=False,
    retrain=False,
    verbose=True,
)

In [ ]:
def eval_backtest(backtest_series, actual_series, horizon, start, transformer):
    plt.figure(figsize=figsize)
    actual_series.plot(label="actual")
    backtest_series.plot(
        low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer
    )
    backtest_series.plot(low_quantile=low_q, high_quantile=high_q, label=label_q_inner)
    plt.legend()
    plt.title(f"Backtest, starting {start}, {horizon}-months horizon")
    print(
        "MAPE: {:.2f}%".format(
            mape(
                transformer.inverse_transform(actual_series),
                transformer.inverse_transform(backtest_series),
            )
        )
    )

In [ ]:
eval_backtest(
    backtest_series=concatenate(backtest_series),
    actual_series=series_transformed,
    horizon=forecast_horizon,
    start=training_cutoff,
    transformer=transformer,
)

# Predicting stocks price with TFT (and make $$)

In [ ]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries

## Getting data

In [ ]:
# !pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
df_stocks = yf.download(tickers=['GOOGL'], period='10y', interval='1d') # , 'AAPL', 'GOOGL'

In [ ]:
df_stocks.head()

In [ ]:
df_stocks.dtypes

In [ ]:
df_stocks = df_stocks.drop('Volume', axis=1)

In [ ]:
alt.Chart(data = df_stocks.reset_index()).mark_line().encode(
    x='Date:T',
    y='Close:Q'
)

## Create 



In [ ]:
# create time series object for target variable
ts_P = TimeSeries.from_series(df_stocks["Close"], fill_missing_dates=True, freq='D') 

In [ ]:
ts_P

In [ ]:
# check attributes of the time series
print("components:", ts_P.components)
print("duration:",ts_P.duration)
print("frequency:",ts_P.freq)
print("frequency:",ts_P.freq_str)
print("has date time index? (or else, it must have an integer index):",ts_P.has_datetime_index)
print("deterministic:",ts_P.is_deterministic)
print("univariate:",ts_P.is_univariate)

In [ ]:
# create time series object for the feature columns
df_covF = df_stocks.loc[:, df_stocks.columns != "Close"]
ts_covF = TimeSeries.from_dataframe(df_covF, fill_missing_dates=True, freq='D')

In [ ]:
# check attributes of the time series
print("components (columns) of feature time series:", ts_covF.components)
print("duration:",ts_covF.duration)
print("frequency:",ts_covF.freq)
print("frequency:",ts_covF.freq_str)
print("has date time index? (or else, it must have an integer index):",ts_covF.has_datetime_index)
print("deterministic:",ts_covF.is_deterministic)
print("univariate:",ts_covF.is_univariate)

In [ ]:
# Define all the hyperparameters

LOAD = False         # True = load previously saved model from disk?  False = (re)train the model
SAVE = "\_TFT_model_02.pth.tar"   # file name to save the model under

EPOCHS = 200
INLEN = 32          # input size
HIDDEN = 64         # hidden layers    
LSTMLAYERS = 2      # recurrent layers
ATTH = 4            # attention heads
BATCH = 32          # batch size
LEARN = 1e-3        # learning rate
DROPOUT = 0.1       # dropout rate
VALWAIT = 1         # epochs to wait before evaluating the loss on the test/validation set
N_FC = 1            # output size

RAND = 1337           # random seed
N_SAMPLES = 100     # number of times a prediction is sampled from a probabilistic model
N_JOBS = 3          # parallel processors to use;  -1 = all processors

# default quantiles for QuantileRegression
QUANTILES = [0.01, 0.1, 0.2, 0.5, 0.8, 0.9, 0.99]

SPLIT = 0.9         # train/test %

FIGSIZE = (9, 6)


qL1, qL2 = 0.01, 0.10        # percentiles of predictions: lower bounds
qU1, qU2 = 1-qL1, 1-qL2,     # upper bounds derived from lower bounds
label_q1 = f'{int(qU1 * 100)} / {int(qL1 * 100)} percentile band'
label_q2 = f'{int(qU2 * 100)} / {int(qL2 * 100)} percentile band'

mpath = os.path.abspath(os.getcwd()) + SAVE     # path and file name to save the mode

In [ ]:
# train/test split and scaling of target variable
ts_train, ts_test = ts_P.split_after(SPLIT)

In [ ]:
print("training start:", ts_train.start_time())
print("training end:", ts_train.end_time())
print("training duration:",ts_train.duration)
print("test start:", ts_test.start_time())
print("test end:", ts_test.end_time())
print("test duration:", ts_test.duration)

In [ ]:
# Scale
scalerP = Scaler()
scalerP.fit_transform(ts_train)

ts_ttrain = scalerP.transform(ts_train)
ts_ttest = scalerP.transform(ts_test)    
ts_t = scalerP.transform(ts_P)

In [ ]:
# make sure data are of type float
ts_t = ts_t.astype(np.float32)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)

In [ ]:
# CHeck first and last row
pd.options.display.float_format = '{:,.2f}'.format
ts_t.pd_dataframe().iloc[[0,-1]]

In [ ]:
# train/test split and scaling of feature covariates
covF_train, covF_test = ts_covF.split_after(SPLIT)

In [ ]:
# Scaler
scalerF = Scaler()
scalerF.fit_transform(covF_train)

covF_ttrain = scalerF.transform(covF_train) 
covF_ttest = scalerF.transform(covF_test)   
covF_t = scalerF.transform(ts_covF) 

In [ ]:
# make sure data are of type float
covF_ttrain = covF_ttrain.astype(np.float32)
covF_ttest = covF_ttest.astype(np.float32)
covF_t = covF_t.astype(np.float32)

In [ ]:
# first and last row of scaled feature covariates
covF_t.pd_dataframe().iloc[[0,-1]]

In [ ]:
# feature engineering - create time covariates: hour, weekday, month, year, country-specific holidays
covT = datetime_attribute_timeseries( ts_P.time_index, attribute="day", add_length=7 )   # 48 hours beyond end of test set to prepare for out-of-sample forecasting
covT = covT.stack(  datetime_attribute_timeseries(covT.time_index, attribute="day_of_week")  )
covT = covT.stack(  datetime_attribute_timeseries(covT.time_index, attribute="month")  )
covT = covT.stack(  datetime_attribute_timeseries(covT.time_index, attribute="year")  )

covT = covT.add_holidays(country_code="US")
covT = covT.astype(np.float32)

In [ ]:
# train/test split
covT_train, covT_test = covT.split_after(ts_train.end_time())

In [ ]:
# rescale the covariates: fitting on the training set
scalerT = Scaler()
scalerT.fit(covT_train)

In [ ]:
covT_ttrain = scalerT.transform(covT_train)
covT_ttest = scalerT.transform(covT_test)
covT_t = scalerT.transform(covT)

In [ ]:
covT_t = covT_t.astype(np.float32)
covT_ttrain = covT_ttrain.astype(np.float32)
covT_ttest = covT_test.astype(np.float32)

In [ ]:
# first and last row of unscaled time covariates:
covT.pd_dataframe().iloc[[0,-1]]

In [ ]:
# combine feature and time covariates along component dimension: axis=1
ts_cov = ts_covF.concatenate( covT.slice_intersect(ts_covF), axis=1 )                      # unscaled F+T
cov_t = covF_t.concatenate( covT_t.slice_intersect(covF_t), axis=1 )                       # scaled F+T
cov_ttrain = covF_ttrain.concatenate( covT_ttrain.slice_intersect(covF_ttrain), axis=1 )   # scaled F+T training set
cov_ttest = covF_ttest.concatenate( covT_ttest.slice_intersect(covF_ttest), axis=1 )       # scaled F+T test set

In [ ]:
# first and last row of unscaled covariates
ts_cov.pd_dataframe().iloc[[0,-1]]

In [ ]:
model = TFTModel(   input_chunk_length=INLEN,
                    output_chunk_length=N_FC,
                    hidden_size=HIDDEN,
                    lstm_layers=LSTMLAYERS,
                    num_attention_heads=ATTH,
                    dropout=DROPOUT,
                    batch_size=BATCH,
                    n_epochs= 5, # EPOCHS,                        
                    nr_epochs_val_period=VALWAIT, 
                    likelihood=QuantileRegression(QUANTILES), 
                    optimizer_kwargs={"lr": LEARN}, 
                    model_name="TFT_stocks",
                    log_tensorboard=True,
                    random_state=RAND,
                    force_reset=True,
                    save_checkpoints=True
                )

In [ ]:
# training: load a saved model or (re)train
if LOAD:
    print("have loaded a previously saved model from disk:" + mpath)
    model = TFTModel.load_model(mpath)                            # load previously model from disk 
else:
    model.fit(  series=ts_ttrain, 
                future_covariates=cov_t, 
                val_series=ts_ttest, 
                val_future_covariates=cov_t, 
                verbose=True)
    print("have saved the model after training:", mpath)
    model.save(path = mpath)

In [ ]:
# testing: generate predictions
ts_tpred = model.predict(   n=len(ts_ttest), 
                            num_samples=N_SAMPLES,   
                            n_jobs=N_JOBS, 
                            verbose=True)

In [ ]:
# retrieve forecast series for chosen quantiles, 
# inverse-transform each series,
# insert them as columns in a new dataframe dfY
q50_RMSE = np.inf
q50_MAPE = np.inf
ts_q50 = None
pd.options.display.float_format = '{:,.2f}'.format
dfY = pd.DataFrame()
dfY["Actual"] = TimeSeries.pd_series(ts_test)

In [ ]:
# helper function: get forecast values for selected quantile q and insert them in dataframe dfY
def predQ(ts_t, q):
    ts_tq = ts_t.quantile_timeseries(q)
    ts_q = scalerP.inverse_transform(ts_tq)
    s = TimeSeries.pd_series(ts_q)
    header = "Q" + format(int(q*100), "02d")
    dfY[header] = s
    if q==0.5:
        ts_q50 = ts_q
        q50_RMSE = rmse(ts_q50, ts_test)
        q50_MAPE = mape(ts_q50, ts_test) 
        print("RMSE:", f'{q50_RMSE:.2f}')
        print("MAPE:", f'{q50_MAPE:.2f}')

In [ ]:
# call helper function predQ, once for every quantile
_ = [predQ(ts_tpred, q) for q in QUANTILES]

In [ ]:
# move Q50 column to the left of the Actual column
col = dfY.pop("Q50")
dfY.insert(1, col.name, col)
dfY.iloc[np.r_[0:2, -2:0]]

In [ ]:
# plot the forecast
plt.figure(100, figsize=(20, 7))
sns.set(font_scale=1.3)
p = sns.lineplot(x = "time", y = "Q50", data = dfY, palette="coolwarm")
sns.lineplot(x = "time", y = "Actual", data = dfY, palette="coolwarm")
plt.legend(labels=["forecast median price Q50", "actual price"])
p.set_ylabel("price")
p.set_xlabel("")
p.set_title("stock price (test set)");